# Homework 3: NBC, KNN, and Ensemble Learning
## CIS600
## Evan Smith

#### Data Loading and EDA
First, we import the data from file.

In [1]:
import pandas as pd

train = pd.read_csv("C:/Users/evana/Documents/GitHub/SyracuseMasters/CIS_600_Fund_Data_Mining/HW3/Disease Prediction Testing.csv")
test = pd.read_csv("C:/Users/evana/Documents/GitHub/SyracuseMasters/CIS_600_Fund_Data_Mining/HW3/Disease Prediction Training.csv")

We then define and use two functions: one to display the null values per row and one for the unique values.

In [5]:
def DisplayNullRows (df):
    dataCleaningCounts = []
    for col in df.columns:
        nullsCount = df[col].isnull().sum()
        if nullsCount > 0:
            dataCleaningCounts.append([col, nullsCount])

    display(pd.DataFrame(dataCleaningCounts, columns = ['Column', 'Nulls']))
    return

def DisplayUniqueVals (df):
    dataCleaningCounts = []
    for col in df.columns:
        uniqueCount = df[col].nunique()
        if uniqueCount > 0:
            dataCleaningCounts.append([col, uniqueCount])

    display(pd.DataFrame(dataCleaningCounts, columns = ['Column', 'Unique Vals']))
    return

DisplayNullRows(train)
DisplayUniqueVals(train)

DisplayNullRows(test)
DisplayUniqueVals(test)

,Column,Nulls


,Column,Unique Vals
0,ID,21000
1,Age,27
2,Gender,2
3,Height,81
4,Weight,182
5,High Blood Pressure,115
6,Low Blood Pressure,92
7,Cholesterol,3
8,Glucose,3
9,Smoke,2


,Column,Nulls


,Column,Unique Vals
0,Age,28
1,Gender,2
2,Height,101
3,Weight,248
4,High Blood Pressure,143
5,Low Blood Pressure,143
6,Cholesterol,3
7,Glucose,3
8,Smoke,2
9,Alcohol,2


We see that there are no null rows in either the test or training set, and all columns contain information (they have at least 2 unique values).

Next, we convert all non-numeric fields such that the various classifiers can process the inputs.